In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [ ]:
# importing required libraries

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import cv2

import sklearn
from sklearn.model_selection import train_test_split

import keras
from keras import callbacks
from keras import optimizers
#from keras.engine import Model
from keras.layers import Dropout, Flatten, Dense, Activation
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint

from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# We first load the necessary libraries, the dataset and reshape its dimensons

# FOR INITIAL RAW IMAGES
#faulty_dir = '/content/drive/Shareddrives/Senior Thesis/Casting_image/casting_512x512/casting_512x512/def_front/'
#ok_dir = '/content/drive/Shareddrives/Senior Thesis/Casting_image/casting_512x512/casting_512x512/ok_front/'

# FOR AUGMENTED IMAGES
faulty_dir = '/content/drive/Shareddrives/Senior Thesis/Casting_image/casting_data/casting_data/images/def_front/'
ok_dir = '/content/drive/Shareddrives/Senior Thesis/Casting_image/casting_data/casting_data/images/ok_front/'
directories = (faulty_dir, ok_dir)


def rescale(i, image, width, height):
  new_im = cv2.resize(image,(width,height))
  # normalize values to be between 0 and 1
  new_im = new_im / 255
  return new_im

# check the total number of images we have
count = 0
for direct in directories:
  for file in os.listdir(direct):
    count += 1

# set the values of the X and y arrays
num_images = count
input_shape = (300, 300, 3)
height = input_shape[0]
width = input_shape[1]
channels = input_shape[2]

X = np.zeros((num_images,height,width,channels))
y = np.zeros((num_images))

# populate the arrays
i = 0
j = -1
for direct in directories:
  j+=1
  for file in os.listdir(direct):
    path = (direct+file)
    im = plt.imread(path)
    scaled_im = rescale(i, im, width, height)
    X[i,:,:,:] = scaled_im
    if j == 0:
      # 1 for faulty
      y[i] = 1
    else:
      # 0 for ok image
      y[i] = 0
    i+=1

# one-hot encoding using function from Keras
y = to_categorical(y)

# use Keras function to split the arrays into training, test, and validation (70%, 15%, 15%)
X_train, X_valid_test, y_train, y_valid_test = train_test_split(X, y, test_size = 0.3, random_state=1)
X_valid, X_test, y_valid, y_test = train_test_split(X_valid_test, y_valid_test, test_size = 0.5, random_state=1)


In [ ]:
print(X_train.shape, X_valid_test.shape, X_valid.shape, X_test.shape)

In [ ]:
# We build the base model
#from keras.applications.vgg16 import VGG16
#base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

from keras.applications import EfficientNetB3
base_model = EfficientNetB3(weights='imagenet', include_top=False, input_shape=input_shape, drop_connect_rate=0.2)

In [ ]:
# We freeze every layer in our base model so that they do not train, we want that our feature extractor stays as before --> transfer learning
for layer in base_model.layers: 
  layer.trainable = False
  print('Layer ' + layer.name + ' frozen.')
# We take the last layer of our the model and add it to our classifier
last = base_model.layers[-1].output
x = Flatten()(last)
x = Dense(1000, activation='relu', name='fc1')(x)
x = Dropout(0.25)(x)
x = Dense(2, activation='softmax', name='predictions')(x)
model = tf.keras.Model(base_model.input, x)

# We compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
# We start the training
mcp_save = ModelCheckpoint('weights.hdf5', save_best_only=True, monitor='val_acc', mode='auto')
model.fit(x=X_train, y=y_train, batch_size=128, epochs=10, verbose=1, callbacks=[mcp_save], validation_data=(X_valid,y_valid), shuffle=True)

In [ ]:
# plot training and validation losses and accuracy versus number of epochs
accuracy = model.history.history['accuracy']
val_accuracy = model.history.history['val_accuracy']
loss = model.history.history['loss']
val_loss = model.history.history['val_loss']
epochs = range(len(accuracy))
plt.plot(epochs, accuracy, 'b', label='Training accuracy')
plt.plot(epochs, val_accuracy, 'r', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
def unfreeze_model(model):
    # We unfreeze the top 10 layers while leaving BatchNorm layers frozen
    for layer in model.layers[-10:]:
        if not isinstance(layer, model.layers.BatchNormalization):
            layer.trainable = True

    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
    model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
unfreeze_model(model)

# We now fine-tune the training
mcp_save = ModelCheckpoint('weights.hdf5', save_best_only=True, monitor='val_acc', mode='auto')
model.fit(x=X_train, y=y_train, batch_size=128, epochs=epochs, verbose=1, callbacks=[mcp_save], validation_data=(X_valid,y_valid), shuffle=True)

In [ ]:
# plot training and validation losses and accuracy versus number of epochs
accuracy = model.history.history['accuracy']
val_accuracy = model.history.history['val_accuracy']
loss = model.history.history['loss']
val_loss = model.history.history['val_loss']
epochs = range(len(accuracy))
plt.plot(epochs, accuracy, 'b', label='Training accuracy')
plt.plot(epochs, val_accuracy, 'r', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()